# 2章 エンドツーエンドの機械学習プロジェクト

[2019年度 西南学院大学経済学部 演習II 講義ノート（担当 市東亘）](http://courses.wshito.com/semi2/2019-bayes-AI/index.html)  
テキスト『scikit-learnとTensorFlowによる実践機械学習』Aurelien Geron著（長尾高弘 訳）オライリー出版


## 2.3 データの準備

ワークスペースに第２章で使うカリフォルニアの住宅価格データセットをコピーする．すでに[別のユーザがKaggleにデータセット](https://www.kaggle.com/harrywang/housing)を追加してくれているのでそれを利用する．

1. 画面右上の「+ADD DATASET」をクリックし，データセットの検索画面を出す．
1. 画面右上の入力欄に「Search Dataset」と薄く表示されている部分に「housing」と入力．
1. 入力すると自動で検索が始まる．検索結果が表示されたら「California Housing Data(1990)」の「Add」ボタンを押す．

これでワークスペースへのデータセットのコピーが開始される．しばらく待つとコピーが完了し，画面右の「Workspace」欄の「input」フォルダにデータセットのフォルダが現れる．

### 2.3.2 データの読み込み

データファイルへのパスは，画面右「Workspace」内の各ファイルをクリックすると，画面下にデータのプレビューが現れ，その一番上にパスが表示される．例えば，`housing.csv`のパスは，`../input/housing.csv`である．

従って，`housing.csv`を読み込む`load_housing_data()`関数（テキストp.44）の`HOUSING_PATH`変数を`../input`に置き換えれば良い．

In [ ]:
import os             # パスの接続にosモジュールを使用
import pandas as pd   # データ操作ライブラリであるpandasモジュールをpdというエイリアスでロード

HOUSING_PATH = "../input"  # データファイルのパスをHOUSING_PATH変数に設定

def load_housing_data(housing_path=HOUSING_PATH):        # load_housing_data関数を定義
    csv_path = os.path.join(housing_path, "housing.csv") # osモジュールを使用してファイルのパスを構築
    return pd.read_csv(csv_path)                         # csvファイルを読み込み

### 2.3.3 データの構造をざっと見てみる

In [ ]:
housing = load_housing_data() # 先ほど定義したload_housing_data関数を実行し結果をhousing変数に読み込む

housing.head()  # head()メソッドで最初の数行を表示

`housing`変数に格納されたデータは，DataFrame型のオブジェクト．

* 各行のデータは区域ごとの観測値．
* 区域の住宅価格中央値 `median_house_value` を様々な属性値を使って予測するのがここの目的．
* 各列の属性一覧
  * `longitude`（経度），`latitude`（緯度），`housing_median_age`（築年数の中央値），`total_rooms`（部屋数），`total_bedrooms`
（寝室数），`population`（人口），`households`（世帯数），`median_income`（収入の中央値），`median_house_value`（住宅価格の中央値），`ocean_proximity`（海との位置関係）の10個．

DataFrameのデータ構造は`info()`メソッドで得られる．（p.45，図2-6）

In [ ]:
housing.info() # DataFrameの構造を表示

* `total_bedrooms`属性だけ非NULLのデータが20,433個しかないことが分かる．
* `ocean_proximity`属性は`float`型ではないので数値データではない．先ほどの`head()`メソッドの結果を見るとカテゴリデータの模様．

In [ ]:
housing['ocean_proximity'].value_counts() # カテゴリ毎のカウント数を表示

DataFrameの数値属性のサマリを得るには`describe()`メソッドを使う．（p.46，図2-7）

In [ ]:
housing.describe()  # DataFrameの統計サマリを表示

各属性の分布をヒストグラムで見てみる．（p.47，図2-8）

In [ ]:
%matplotlib inline
# 上の1行はJupyterノートブックでのみ必要
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()  # Jupyterノートブックではhist()メソッドが直接描画を出力するので，show()メソッドはなくてもOK

### 2.3.4 テスト・データセットの作成

データ・スヌーピング・バイアス（data snooping bias）を避けるため，データは最低でも訓練データとテストデータに分ける必要がある．テキストp.30には訓練データ80%，テストデータ20%に分けるとあるが，実際には学習器の訓練中（アルゴリズムやハイパーパラメータの選定）のモデル評価に使うデータと，最終的な予測性能の見積もりに使うデータも分けるのが望ましい．一般には，学習器の訓練中に使う「訓練データセット（training dataset）」と，訓練中のモデル改善評価に使う「検査データセット（validation dataset）」，最後に１回だけ行うモデル性能評価に使う「テストデータセット（test dataset）」の３つに分割する．その割合は順に，50%，25%，25%に分割するのが一般的である．

データセットを構築する際に，クラスに極端な偏りがあり無作為抽出では全データセットのクラス分布を正しく反映できない場合は，層化（無作為）抽出（stratified random sampling）を使う．

以下ランダム抽出と層化抽出の順に解説する．

### ランダム抽出

以下で定義する`split_data()`関数は，第2引数の`test_ratios`に分割する比率を指定すると，分割されたデータを配列で返す．ランダム抽出を再現する場合は，第3引数の`seed`に数字を指定する．デフォルトでは，0.5, 0.25, 0.25の3つに分割する．

In [ ]:
import numpy as np

def split_data (data, test_ratios=[0.5, 0.25, 0.25], seed=None):
    if sum(test_ratios) != 1.0:
        print("test_ratios must sum up to 1.0")
        return
    if seed:
        np.random.seed(seed)
    shuffled_indices = np.random.permutation(len(data))
    accum = 0
    beg = 0
    result = []
    size = len(data)
    for i, v in enumerate(test_ratios):
        accum += v
        to = int(size*accum)
        result.append(data.iloc[shuffled_indices[beg:to]])
        beg = to
    return result

使用例を以下に示す．Pythonでは配列を左辺に代入する際，複数の変数にunpackingしてくれる．

In [ ]:
training, validation, test = split_data(housing, [0.5, 0.3, 0.2])
print(len(training), ", ", len(validation), ", ", len(test))
print(len(training) + len(validation)+  len(test), "==", len(housing))



In [ ]:
training, validation, test = split_data(housing, [0.5, 0.25, 0.25], 1234) # seedを指定
training.head()

In [ ]:
training2, validation2, test2 = split_data(housing, [0.5, 0.25, 0.25], 1234) # seedを指定
training2.head()

### 層化抽出

収入の中央値，`median_income`列をベースに層化抽出する方法を示す．層化する前に`median_income`の分布をヒストグラムで見てみる．

In [ ]:
housing.plot(y=["median_income"], kind='hist')

In [ ]:
housing["median_income"].max()

最大値が15.0001．これを5段階の層に分割するために，データの値を圧縮する．最高値を5の値に圧縮するには`1/3`倍すれば良いが，そのまま圧縮すると高額所得の頻度が少ないままになるので`2/3`倍し，5以上の値は全て5番目の層に分類することにする．

In [ ]:
housing["income_cat"] = np.ceil(housing["median_income"] * 2.0 / 3.0) # 圧縮した値を新たにincome_cat列に追加
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True) # where(条件, 条件Falseの時の値か処理, 置き換えるか否か)

In [ ]:
housing.plot(y=["income_cat"], kind='hist')

上のヒストグラムによりうまく5段階に分類できた．`income_cat`の値に応じて層化抽出する．層化抽出するには scikit-learn の`StratifiedShuffleSplit`クラスを使う．`StratifiedShuffleSplit`はデータを2つに分割することしかできないので，0.5, 0.25, 0.25に分割したければ，最初に0.5で分割して，片方をさらに0.5で分割すれば良い．`n_splits`引数は今は1を指定する．この引数はCross Validationという標本再抽出を行う際に使用する．`random_state`引数は再現性のある分割を行う場合の乱数シードの指定である．

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# 1回目の1/2分割で訓練データセットが確定
split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for train_index, half_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    half = housing.loc[half_index]
# 2回目の1/2分割で検査データセットとテストデータセットを作成
split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=99)
for test_index, validation_index in split.split(half, half["income_cat"]):
    strat_validation_set = housing.loc[validation_index]
    strat_test_set = housing.loc[test_index]

    

各データセットの分布を確認する．

In [ ]:
strat_train_set.plot(y=["income_cat"], kind='hist')

In [ ]:
strat_validation_set.plot(y=["income_cat"], kind='hist')

In [ ]:
strat_test_set.plot(y=["income_cat"], kind='hist')

## 2.5 データクリーニングとデータ整形

区域の住宅価格中央値 `median_house_value` を様々な属性値を使って予測するのがここの目的．この予測器を生成する機械学習アルゴリズムを実装した関数が要求するデータの形式にデータを整形する．

- ターゲット: 住宅価格中央値 `median_house_value`
- 予測子（predictors）: 住宅価格に影響を及ぼしそうな様々な属性データ．

まず，`housing`変数に予測子のみからなるデータフレームを，`housing_labels`変数にターゲットの値（予測の正解値）を格納しておく．

In [ ]:
# median_house_value列と層化抽出のために一時的に作成したincome_cat列を取り除いたデータフレームのコピーを返す
housing = strat_train_set.drop(["median_house_value", "income_cat"], axis=1)

# median_house_value列データのコピーを返す
housing_labels = strat_train_set["median_house_value"].copy()

### 2.5.1 欠損値の処理

欠損値があると機械学習アルゴリズムで問題が生じるのでデータフレーム内の欠損値の有無をチェックする．

データフレームの`isnull()`メソッドはデータフレームの各セルの結果を`True`，`False`としてデータフレームと同じ形（次元）のデータ構造で返す．`any()`メソッドは列方向に真偽値の論理和を返す．行方向に見たいい場合は`any(axis=1)`を使う．

In [ ]:
housing.isnull().any()

`total_bedrooms`列に欠損値が含まれているのがわかる．欠損値のあるデータ数をカウントしてみる．

In [ ]:
housing.isnull().sum()

In [ ]:
housing.shape

10,320個のデータ中，107個の欠損値があることがわかる．



**続く...**